In [ ]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import spacy 
import optuna
import xgboost as xgb
import re
from spacy_langdetect import LanguageDetector
from spacy.language import Language

import sys; sys.path.insert(0, '../') #adds all the code we've written in src
from tqdm.notebook import trange, tqdm
from preprocessing import *

from sklearnex import patch_sklearn
patch_sklearn()

from sklearnex.svm import SVC
from sklearn.metrics import accuracy_score as accuracy
from sklearn.linear_model import LogisticRegression as LR
from sklearn.metrics import classification_report 
from sklearn.metrics import accuracy_score as accuracy
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer as tfidf
from sklearn.preprocessing import MaxAbsScaler

In [ ]:
@Language.factory('language_detector')
def language_detector(nlp, name):
    return LanguageDetector()

nlp = spacy.load("en_core_web_sm", disable = ["tagger", "parser", "attribute_ruler", "ner"])
nlp.add_pipe('sentencizer')
nlp.add_pipe('language_detector', last=True) #2
spacy_tokenizer = nlp.tokenizer
docs = nlp.pipe(tweets)

In [ ]:
def get_number_of_ats(tweet): 
    at_regex = r"@"
    ats = re.findall(at_regex, tweet)
    return len(ats)

def get_number_of_hashtags(tweet): 
    return len(re.findall(r"#", tweet))

def is_retweet(tweet): 
    return len(re.findall(r"rt", tweet))

df_en["num_ats"] = df_en.tweet_text.apply(lambda row: get_number_of_ats(row))
df_en["num_tags"] = df_en.tweet_text.apply(lambda row: get_number_of_hashtags(row))
df_en["rt"] = df_en.tweet_text.apply(lambda row: is_retweet(row))

cols = ["cyberbullying_type", "num_ats", "score", "num_tags", "rt"]
df_en[cols].groupby("cyberbullying_type").mean()